1. Load all the volcano plot files, also the TF file
2. For all the files, filter rows based on padj <0.01
3. Next, filter rows based on the transcriptional factor genes
3. Based upon the filtering, create the final set of ensemble gene ids 
4. Filter the countData file rows based upon the final set of ensemble gene ids - send the final file to musaddiq

In [1]:
# Load libraries

import os
import pandas as pd

In [2]:
# Read the TF file

df_tffile = pd.read_csv('../Rattus_norvegicus_TF.txt', sep='\t')
tf_list = list(df_tffile['Ensembl'])
df_tffile.head()

,Species,Symbol,Ensembl,Family,Protein,Entrez_ID
0,Rattus_norvegicus,Prdm2,ENSRNOG00000033522,zf-C2H2,ENSRNOP00000046011.3;,313678.0
1,Rattus_norvegicus,NaN,ENSRNOG00000067812,zf-C2H2,ENSRNOP00000086758.1;ENSRNOP00000089484.1;,102551257.0
2,Rattus_norvegicus,Isl2,ENSRNOG00000015336,Homeobox,ENSRNOP00000021074.3;,57233.0
3,Rattus_norvegicus,Fos,ENSRNOG00000008015,TF_bZIP,ENSRNOP00000010712.2;,314322.0
4,Rattus_norvegicus,Vax1,ENSRNOG00000008824,Homeobox,ENSRNOP00000011743.1;,64571.0


In [3]:
# Create a method to preprocess the deg files

def tf_filter(df):
    df = df[df['padj'] <= 0.01]
    df = df[df['ENSEMBL ID'].isin(tf_list)]
    # df = df[abs(df['log2FoldChange']) >= 0.5]
    print(f'No of rows: {len(df)}')

    return list(df['ENSEMBL ID'])

In [18]:
# Read all the differential gene expression files

path = '../'
deg_files = [f for f in os.listdir(path) if f.endswith('.csv')]

gene_list_all = []
for deg_file in deg_files:
    df = pd.read_csv(os.path.join(path, deg_file))
    gene_list = tf_filter(df)
    gene_list_all.append(set(gene_list))

# gene_list_union = set.union(*gene_list_all)
gene_list_intersection = set.intersection(*gene_list_all)

print(len(gene_list_intersection))
# list(gene_list_intersection)

No of rows: 205
No of rows: 548
No of rows: 503
No of rows: 495
No of rows: 580
No of rows: 454
No of rows: 302
No of rows: 262
No of rows: 347
No of rows: 454
37


In [15]:
# Load the all-count file

df_all_count = pd.read_csv('../../data/A_FullClean_CountData.csv')
df_all_count.head()

,ID,TCPS.1,TCPS.2,TCPS.3,OM.1,OM.2,OM.3,BMP2.1,BMP2.2,BMP2.3,PT.1,PT.2,PT.3,mSLA.1,mSLA.2,mSLA.3,Titan.1,Titan.2,Titan.3
0,ENSRNOG00000046319,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,ENSRNOG00000050370,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,ENSRNOG00000040300,10,11,10,10,12,7,6,10,17,12,24,13,31,46,52,27,45,42
3,ENSRNOG00000058808,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,ENSRNOG00000061316,834,963,894,619,598,630,1058,839,962,1409,1436,844,2364,2237,2736,3461,4492,3960


In [20]:
# Filter the all-count dataframe based on gene list intersection

df_all_count = df_all_count[df_all_count['ID'].isin(gene_list_intersection)]
# df_all_count.to_csv('../../data/CountData_TF_union.csv', index=False)
df_all_count.to_csv('../../data/CountData_TF_intersection.csv', index=False)
df_all_count.head()

,ID,TCPS.1,TCPS.2,TCPS.3,OM.1,OM.2,OM.3,BMP2.1,BMP2.2,BMP2.3,PT.1,PT.2,PT.3,mSLA.1,mSLA.2,mSLA.3,Titan.1,Titan.2,Titan.3
699,ENSRNOG00000020426,4258,4973,4691,2346,2164,2520,5947,4859,5887,4879,5028,3065,4716,4793,4915,3892,4255,4384
856,ENSRNOG00000020762,5652,6572,6561,4952,4756,5301,5876,5026,5887,5340,5842,3488,4885,4539,5186,4260,4768,4698
1173,ENSRNOG00000022537,118,97,106,74,49,78,134,117,154,286,260,162,436,427,484,646,920,770
1383,ENSRNOG00000013017,197,227,215,1093,1040,1146,185,178,197,181,198,117,143,110,137,66,89,72
2218,ENSRNOG00000020552,800,989,863,517,507,596,826,677,825,1686,1575,828,2617,2612,2750,5504,6071,5855
